In [39]:
import neat
import os
import evogym.envs
from evogym import is_connected, has_actuator, get_full_connectivity, hashable
import numpy as np
from pathos.multiprocessing import ProcessPool
import errno
import dill
import sys
sys.path.append('../')

import neat
from hyperneat.hyperNEAT import create_phenotype_network
from hyperneat.substrate import Substrate
from hyperneat.visualize import draw_net
from dynamic_env.traverser import DynamicObstacleTraverser

import itertools as it
import time
import imageio

import sgr_main as sgr
from matplotlib import pyplot as plt


from sgr.substrates import morph_substrate, control_substrate
from sgr.generate_robot import generate_robot
from sgr.generate_robot import eval_robot_constraint

from sgr.evogym_sim import simulate_env
from sgr.body_speciation import CustomGenome


In [31]:
ENV = "Climber-v0" # env_names = ["CaveCrawler-v0", "UpStepper-v0", "ObstacleTraverser-v1"]
STEPS = 600
ROBOT_SIZE = 5
POP = 128

# directory = "results/" + ENV
directory = "checkpoints/"

file = "climber_pop_gen_19.pkl"
params = {"robot_size": ROBOT_SIZE, "env": ENV, "steps": STEPS, "pop_size": POP, "robot_substrate": "3d"}

CustomGenome.params = params
CustomGenome.robot_func = lambda self, net, config: generate_robot(net, params["robot_size"])
CustomGenome.substrate = morph_substrate(params["robot_size"], params["robot_substrate"])
CustomGenome.robot_size = params["robot_size"]
CustomGenome.spec_genotype_weight = 1
CustomGenome.spec_phenotype_weight = 2

neat_config = neat.config.Config(
    CustomGenome,
    neat.reproduction.DefaultReproduction, 
    neat.species.DefaultSpeciesSet, 
    neat.stagnation.DefaultStagnation, 
    "../neat_configs/hyperNEAT.cfg"
)

input_size = 4*2 + 1 # two coordinates plus the bias
neat_config.genome_config.num_inputs = input_size
neat_config.genome_config.input_keys = [-1*i for i in range(1, input_size+1)]
neat_config.genome_config.num_outputs = 1
neat_config.genome_config.output_keys = [0]

with open(f"../{directory}/{file}", "rb") as file:
    pop = dill.load(file)
champion = pop.best_genome
print(champion.fitness)

9.253902323200013


In [50]:
def get_robot(genome, params, neat_config):
    cppn = neat.nn.FeedForwardNetwork.create(genome, neat_config)
    design_substrate = morph_substrate(params["robot_size"], params["robot_substrate"])
    design_net = create_phenotype_network(cppn, design_substrate)

    robot = generate_robot(design_net, params["robot_size"], params["robot_substrate"])
    print(robot)
    if not eval_robot_constraint(robot):
        return robot, None
    connections = get_full_connectivity(robot)
    if params["env"] == "dynamic":
        env = DynamicObstacleTraverser(body=robot, connections=connections)
    else:
        env = evogym.envs.gym.make(params["env"], body=robot, connections=connections)
    img = env.render(mode='img')
    env.close()
    return robot, img

In [51]:
for k, v in pop.species.species.items():
    # print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Specie ", k, "!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    
    for g in v.members.values():
        if g.fitness != None and g.fitness >4:
            
            print(g.fitness)
            r, img = get_robot(g, params, neat_config)
            if img is not None:
                print(r)
                print(g.key, g.fitness)
                plt.imshow(img)
                plt.show()
                net = neat.nn.FeedForwardNetwork.create(g, neat_config)
                display(draw_net(net))

9.253902323200013
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [42]:
g = pop.best_genome
# g = pop.population[14774]
params["save_to"] = "walker"
sgr.single_genome_fit(g, params, neat_config, True, False)
# sgr.single_genome_fit(g, params, neat_config, False, True)

AttributeError: module 'sgr_main' has no attribute 'single_genome_fit'

In [6]:
base = pop.population[12892]
r2 = pop.population[14520]

_, img = get_robot(base, params, neat_config)
plt.imshow(img)
plt.show()
net = neat.nn.FeedForwardNetwork.create(base, neat_config)
display(draw_net(net))

_, img = get_robot(r2, params, neat_config)
plt.imshow(img)
plt.show()
net = neat.nn.FeedForwardNetwork.create(r2, neat_config)
display(draw_net(net))


neat_config.compatibility_disjoint_coefficient = 1
neat_config.compatibility_weight_coefficient = .6

base.distance = f
print(base.distance(base, r2, neat_config))


print()

base.distance =  neat.DefaultGenome.distance
print(base.distance(base, r2, neat_config))


KeyError: 12892